### Setup

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'AVRCRYPTOLIB'

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../../hardware/victims/firmware/simpleserial-rsa
#make clean PLATFORM=$1 CRYPTO_TARGET=$2
make PLATFORM=$1 CRYPTO_TARGET=$2

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = '../../../hardware/victims/firmware/simpleserial-rsa/simpleserial-rsa-CWLITEXMEGA.hex'

In [ ]:
cw.program_target(scope, prog, fw_path)

In [ ]:
import numpy as np
import time 
import matplotlib.pylab as plt
import matplotlib

In [ ]:
e = 65537
n = 52270967183635299110959169812507925945033892524340745668343932985606720718177

### find numbers

In [ ]:
import random
from sympy import nextprime

#generate 16 byte number 
unsigned_16_byte = random.randint(0, 2**128 - 1)
print(unsigned_16_byte)



prime1 = nextprime(unsigned_16_byte)
print("p1:", prime1)
print(prime1.to_bytes(16,'big'))

#p1: 195126513202591094461357355156264195281

#p2: 267882443680857975761967467408531606417


In [ ]:
# prime numbers with 32 bytes 
#p1: 195126513202591094461357355156264195281

#p2: 267882443680857975761967467408531606417


p = 267882443680857975761967467408531606417
pb = p.to_bytes(16,'big')
print('p', p, pb, hex(p))
q = 195126513202591094461357355156264195281
qb = q.to_bytes(16,'big')
print('q', q, qb, hex(q))

N = p*q
#print(N) # 52270967183635299110959169812507925945033892524340745668343932985606720718177
Nb = N.to_bytes(32,'big')
print('N', N, Nb, hex(N))

e = 65537
eb = e.to_bytes(16,'big')
print('e', e, eb, hex(e))

euler = (p-1)*(q-1)
print(euler)
# 52270967183635299110959169812507925944570883567457296598120608163041924916480

#imversis zu e mod euler: e*d + k * euler = 1 = ggT(e, euler)
# www.ardt-brunner.de : 
# -23185635839728369396761875985315247985240026020507249524808369002237343139327
#·65537 
#+ 29070
#·52270967183635299110959169812507925944570883567457296598120608163041924916480 = 1


In [ ]:
d_ = -23185635839728369396761875985315247985240026020507249524808369002237343139327


d = d_ + euler
# 29085331343906929714197293827192677959330857546950047073312239160804581777153
db = d.to_bytes(32,'big')
print('d', d, db, hex(d))

#test
#t = (d * e ) % euler
# print(t)

dp = d % (p-1)
#print(dp)
#256641820519554293363545041368684467889
dpb = dp.to_bytes(16,'big')
print('dp', dp, dpb, hex(dp))

dq = d % (q-1)
#185161326518899860606250116989915167073
dqb = dq.to_bytes(16,'big')
print('dq', dq, dqb, hex(dq))


qinv = pow(q, -1, p)
#152353370281789692140390938043637092096
qinvb = qinv.to_bytes(16,'big')
print('qinv', qinv, qinvb, hex(qinv))

#test
#t2 =(qinv * q) % p
#print(t2)

### capture trace setup

In [ ]:
scope.clock.adc_src = "clkgen_x1"
#scope.adc.samples = 1000
scope.adc.samples = 100

#scope.adc.timeout = 

In [ ]:
import time

def capture_RSA_trace(scope, target, text):
    scope.arm()
    target.simpleserial_write('h', text)

    # get debug output
    rec_str = target.read(1000, timeout=5000) #ohne debug
    #rec_str = target.read(2000, timeout=15000) #mit debug
    received_str = rec_str.replace("\\r\\n", "\n")
    print(received_str)
    
    ret = scope.capture()
    if ret:
        return None
    target.simpleserial_wait_ack()
    return scope.get_last_trace()

### test numbers for 16 Byte prim numbers

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 0
c = (msg ** e) % n

msg2_d1 = c.to_bytes(32,'big')

print(c, msg2_d1, type(msg2_d1))

trace0 = capture_RSA_trace(scope, target, msg2_d1)
plt.figure()
plt.plot(trace0, 'r')

In [ ]:
%matplotlib ipympl
import matplotlib.pylab as plt

msg = 1
c = (msg ** e) % n

msg1 = c.to_bytes(32,'big')

print(c, msg1, type(msg1))

trace1 = capture_RSA_trace(scope, target, msg1)
plt.figure()
plt.plot(trace1, 'r')

In [ ]:
msg = bytearray([0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x67, 0x0c])
trace_d1 = capture_RSA_trace(scope, target, msg)
plt.figure()
plt.plot(trace_d1, 'r')
#diff(x) = 1

#35 sek

In [ ]:


msg = bytearray([0x34, 0xd1, 0x5f, 0x38, 0x73, 0xc4, 0xf7, 0x1d, 0x4e, 0x92, 0x00, 0x32, 0x5e, 0xdc, 0x81, 0xe8, 0x40, 0xd2, 0x32, 0x67, 0x99, 0x85, 0x83, 0x0d, 0x21, 0x00, 0xf8, 0xfd, 0x90, 0xe5, 0x25, 0x95])
trace_d1 = capture_RSA_trace(scope, target, msg)
plt.figure()
plt.plot(trace_d1, 'r')
#diff(x) = 0

#expected output:
#DBG: expmod m1 ...
#expmod(34d15f3873c4f71d4e9200325edc81e840d232679985830d2100f8fd90e52595,c1136aada4591b8d8890e08a9a4096b1,c98848303d0609bd390bfeebe5ef4b91) = 
#+e000000000000000expmod m2 ...
#expmod(34d15f3873c4f71d4e9200325edc81e840d232679985830d2100f8fd90e52595,8b4cc63a64dac10f923127f33366f961,92cbffdd66dfbe4fc1b267f7389c14d1) = 
#+e000000000000000
#DBG: sub ...
#sub(e000000000000000,e000000000000000) = 0
#DBG: reduce-mul ...
#reduce(0,c98848303d0609bd390bfeebe5ef4b91) = 0
#mul(0,729e311980e10a45ae39bb48045ccf00) = 0
#reduce(0,c98848303d0609bd390bfeebe5ef4b91) = 0
#mul(0,92cbffdd66dfbe4fc1b267f7389c14d1) = 0
#add(0,e000000000000000) = e000000000000000z00

In [ ]:
%matplotlib ipympl


msg = 62558
c = (msg ** e) % n

msg2_d1 = c.to_bytes(32,'big')

print(c, msg2_d1, type(msg2_d1))

trace2_d1 = capture_RSA_trace(scope, target, msg2_d1)
plt.figure()
plt.plot(trace2_d1, 'r')

In [ ]:
print(np.sum(np.abs(trace_d1 - trace1)))
print(np.sum(np.abs(trace2_d1 - trace1)))
print(np.sum(np.abs(trace_d1 - trace2_d1)))

In [ ]:
plt.close('all')

In [ ]:
matplotlib.pyplot.close()

### Key Reconstuction

* find plaintexts x_1 and x_2 mit diff(x_1) = 0 und diff(x_2) = 1.
* mittels binäre suche: finde x mit diff(x-1) = 0 und diff(x) = 1 -> x ist Vielfaches von p 
* ggt(x, n) = p
* x^e mod n

### Algorithmus

Reconstruction of p when its bitlength is half bitlength of n
INPUT: modulus n with the bitlength t, public exponent e.
OUTPUT: prime p such that p divides n.

t = bitlenght von n 
p,q haben bitlänge t/2

x = 2 ** (t/2)
m = x/2 

werte liegen bei 2 und bei 12 -> mitte  wählen bei mit debug 
werte liegen bei 2 und bei 11 -> mitte ist 7/6

In [ ]:
# 16 Byte Primzahlen 
#16*8=128
# 32 * 8 = 256

t = 256

# x= 2 ** (t/2) 
# m = x/2 
# l = 0 

x = 2**128
m = x//2
l = 0

print(x, m, l)

In [ ]:

 
plt.figure()

ref_trace_msg_ = 1 # loop wird nicht ausgeführt, diff(m) = 0
c_ = (ref_trace_msg_ ** e) % n 
msg_ = c_.to_bytes(32,'big')
trace_ = capture_RSA_trace(scope, target, msg_)


while m != l:
    #enc = (m ** e) % n
    
    print(m, type(m), '\n')
    enc = pow(m, e, n)
    msg = enc.to_bytes(32,'big')
    
    trace = capture_RSA_trace(scope, target, msg)
    plt.plot(trace)

    diff = np.sum(np.abs(trace - trace_))
    print("{:1} diff = {:2}".format(m, diff))
    
    #check if diff(m) = 1
    # debug: sad von d1 und d0 = 6  und sad von d0 und d0 = 2-> wähle diff 4
    # no debug sad von d1 und d0 = 7 und sad von d0 und d0 = 3,5 -> wähle diff 5/6
    if diff < 4: # ähnliche -> diff(m) = 0
        l = m
    else: #unterscheidet sich von ref_trace -> diff(m) = 1
        x = m
        
    m = (l + x)//2
    
    

print (x)
    

In [ ]:
#output: 18446744073709551616-> x 
     
#q: 195126513202591094461357355156264195281

#p: 267882443680857975761967467408531606417

In [ ]:
scope.dis()
target.dis()

laufzeit mit debug: 1h
lauzeit ohne debug: 40 min